<a href="https://colab.research.google.com/github/Kotwic4/fake-news-detector/blob/colab/colab/autokeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests numpy matplotlib pandas git+git://github.com/keras-team/autokeras@master#egg=autokeras

  Cloning git://github.com/keras-team/autokeras (to revision master) to /tmp/pip-install-n8pyg77u/autokeras
  Running command git clone -q git://github.com/keras-team/autokeras /tmp/pip-install-n8pyg77u/autokeras
     |████████████████████████████████| 86.3MB 122kB/s 
     |████████████████████████████████| 92kB 13.5MB/s 
     |████████████████████████████████| 3.8MB 54.7MB/s 
     |████████████████████████████████| 450kB 63.2MB/s 
     |████████████████████████████████| 81kB 12.7MB/s 
  Created wheel for autokeras: filename=autokeras-1.0.0a0-cp36-none-any.whl size=50331 sha256=7a1fc09a5afcacf4478449b58f71679f79f870993cd85d614bd29f0bfdb08c15
  Stored in directory: /tmp/pip-ephem-wheel-cache-dvja_qyl/wheels/55/b0/bc/7304f4fcafa1a7f163e4e4da7cb951f17b1cacf2e76b7d3146
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15354 sha256=468177694ff532878c59e00406bf1964eb1c5c15eb503b900878a1d11c8636b2
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6

In [0]:
!nvidia-smi

Tue Dec 10 21:40:42 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!wget - O politifact_fake.csv https://raw.githubusercontent.com/KaiDMML/FakeNewsNet/master/dataset/politifact_fake.csv
!wget - O politifact_real.csv https://raw.githubusercontent.com/KaiDMML/FakeNewsNet/master/dataset/politifact_real.csv

--2019-12-10 21:40:48--  http://-/
Resolving - (-)... failed: Name or service not known.
wget: unable to resolve host address ‘-’
--2019-12-10 21:40:48--  http://o/
Resolving o (o)... failed: Name or service not known.
wget: unable to resolve host address ‘o’
--2019-12-10 21:40:48--  http://politifact_fake.csv/
Resolving politifact_fake.csv (politifact_fake.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘politifact_fake.csv’
--2019-12-10 21:40:48--  https://raw.githubusercontent.com/KaiDMML/FakeNewsNet/master/dataset/politifact_fake.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3286418 (3.1M) [text/plain]
Saving to: ‘politifact_fake.csv.1’

politifact_fake.csv 100%[===================>]   3.13M  --.-KB/s    in 0.08s   

2019

In [0]:
!ls -l

total 22608
-rw-r--r--  1 root root 3286418 Dec 10 20:38 politifact_fake.csv
-rw-r--r--  1 root root 3286418 Dec 10 21:40 politifact_fake.csv.1
-rw-r--r--  1 root root 8278658 Dec 10 20:38 politifact_real.csv
-rw-r--r--  1 root root 8278658 Dec 10 21:40 politifact_real.csv.1
drwxr-xr-x  1 root root    4096 Dec  6 16:53 sample_data
drwxr-xr-x 15 root root    4096 Dec 10 21:39 text_classifier


In [0]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd
import os
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
from autokeras import TextClassifier

In [0]:
DATASET_NAME = 'politifact'
DATASET_PATH = './{}'.format(DATASET_NAME)
REAL_DATA_PATH = '{}_real.csv'.format(DATASET_PATH)
FAKE_DATA_PATH = '{}_fake.csv'.format(DATASET_PATH)

In [0]:
fake_arts = pd.read_csv(FAKE_DATA_PATH, na_values=['nan'], keep_default_na=False)
real_arts = pd.read_csv(REAL_DATA_PATH, na_values=['nan'], keep_default_na=False)

In [0]:
def parse_art_data_frame(df):
    return [{'id': id, 'url': url, 'title': title} for id, url, title, tweets in df.values]
    
fake_arts_with_content = parse_art_data_frame(fake_arts)
real_arts_with_content = parse_art_data_frame(real_arts)

In [0]:
fake_data = [(art, 'fake') for art in fake_arts_with_content]
real_data = [(art, 'real') for art in real_arts_with_content]

In [0]:
all_data = fake_data + real_data
X = [x for x,y in all_data]
y = [y for x,y in all_data]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_data = list(zip(X_train, y_train))
test_data = list(zip(X_test, y_test))

In [0]:
print(len(train_data))
print(len(test_data))
print(len(train_data) + len(test_data))
print(len(fake_data) + len(real_data))

844
212
1056
1056


In [0]:
print(len([y for y in y_train if y == 'fake']))
print(len([y for y in y_test if y == 'fake']))

print(len([y for y in y_train if y == 'real']))
print(len([y for y in y_test if y == 'real']))

334
98
510
114


In [0]:
def clear_text(text):
    return ' '.join(text.split())

class Classifier():
    def __init__(self, classifier):
        self.classifier = classifier
        
    def predict(self, text):
        text = clear_text(text)
        sentence = Sentence(text)
        self.classifier.predict(sentence)
        return sentence.labels[0]
    
def train_model(train_data,
               data_folder = '.',
               model_folder = '.',
               max_epochs=1
               ):
    clf = TextClassifier(verbose=True)
    train_data
    clf.fit(x=x_train, y=y_train)
    return Classifier(clf)

def calculate_metrics(y_true, y_pred, pos_label = 'fake'):
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, pos_label=pos_label)
    recall = recall_score(y_true, y_pred, pos_label=pos_label)
    f1 = f1_score(y_true, y_pred, pos_label=pos_label)
    return acc, precision, recall, f1

def validate_model(test_data, classifier):
    y_true = [label for x, label in test_data]
    y_pred = [classifier.predict(x).value for x, label in test_data]
    acc, precision, recall, f1 = calculate_metrics(y_true, y_pred)
    print("acc: ", acc)
    print("precision: ", precision)
    print("recall: ", recall)
    print("f1: ", f1)
    return acc, precision, recall, f1

def make_test(train_data, test_data, data_folder, model_folder, max_epochs):
    classifier = train_model(train_data, data_folder, model_folder, max_epochs)
    validate_model(test_data, classifier)

In [0]:
train_content = [(x, label) for x, label in train_data] 
test_content = [(x, label) for x, label in test_data]

In [0]:
train_title = [(x['title'], label) for x, label in train_content]
test_title = [(x['title'], label) for x, label in test_content]
print(len([x for x, label in train_title if x == '']))
print(len([x for x, label in test_title if x == '']))
train_title = [(x, label) for x, label in train_title if x != ''] 
test_title = [(x, label) for x, label in test_title if x != '']
print(len(train_title))
print(len(test_title))

0
0
844
212


In [0]:
train_url = [(x['url'], label) for x, label in train_content] 
test_url = [(x['url'], label) for x, label in test_content]
print(len([x for x, label in train_url if x == '']))
print(len([x for x, label in test_url if x == '']))
train_url = [(x, label) for x, label in train_url if x != ''] 
test_url = [(x, label) for x, label in test_url if x != '']
print(len(train_url))
print(len(test_url))

39
22
805
190


In [0]:
train_mix = [(x['url'] + ', ' + x['title'], label) for x, label in train_content] 
test_mix = [(x['url'] + ', ' + x['title'], label) for x, label in test_content]
print(len([x for x, label in train_mix if x == '']))
print(len([x for x, label in test_mix if x == '']))
train_mix = [(x, label) for x, label in train_mix if x != ''] 
test_mix = [(x, label) for x, label in test_mix if x != '']
print(len(train_mix))
print(len(test_mix))

0
0
844
212


In [0]:
x_train, y_train = zip(*train_mix)
x_train = np.array(x_train)
y_train = np.array(y_train)
clf = TextClassifier(max_trials=50)
clf.fit(x=x_train, y=y_train, validation_split=0.2, epochs=50)

Epoch 1/50
22/22 [==============================] - 2s 92ms/step - loss: 0.6758 - accuracy: 0.6050 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
22/22 [==============================] - 1s 53ms/step - loss: 0.6178 - accuracy: 0.6036 - val_loss: 0.5855 - val_accuracy: 0.6190
Epoch 3/50
22/22 [==============================] - 1s 54ms/step - loss: 0.4991 - accuracy: 0.7574 - val_loss: 0.4604 - val_accuracy: 0.8214
Epoch 4/50
22/22 [==============================] - 1s 52ms/step - loss: 0.3445 - accuracy: 0.9453 - val_loss: 0.3334 - val_accuracy: 0.8929
Epoch 5/50
22/22 [==============================] - 1s 49ms/step - loss: 0.2088 - accuracy: 0.9822 - val_loss: 0.2418 - val_accuracy: 0.9286
Epoch 6/50
22/22 [==============================] - 1s 57ms/step - loss: 0.1175 - accuracy: 0.9926 - val_loss: 0.1910 - val_accuracy: 0.9345
Epoch 7/50
22/22 [==============================] - 1s 55ms/step - loss: 0.0663 - accuracy: 0.9970 - val_loss: 0.1686 - val_accuracy: 0.9345
Epoch

Epoch 1/50
22/22 [==============================] - 2s 87ms/step - loss: 0.6960 - accuracy: 0.3994 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
22/22 [==============================] - 1s 59ms/step - loss: 0.6956 - accuracy: 0.4009 - val_loss: 0.6959 - val_accuracy: 0.3690
Epoch 3/50
22/22 [==============================] - 1s 57ms/step - loss: 0.6956 - accuracy: 0.4009 - val_loss: 0.6959 - val_accuracy: 0.3750
Epoch 4/50
22/22 [==============================] - 1s 57ms/step - loss: 0.6956 - accuracy: 0.4024 - val_loss: 0.6959 - val_accuracy: 0.3750
Epoch 5/50
22/22 [==============================] - 1s 59ms/step - loss: 0.6956 - accuracy: 0.4024 - val_loss: 0.6959 - val_accuracy: 0.3750
Epoch 6/50
22/22 [==============================] - 1s 55ms/step - loss: 0.6956 - accuracy: 0.4024 - val_loss: 0.6959 - val_accuracy: 0.3750
Epoch 7/50
22/22 [==============================] - 1s 56ms/step - loss: 0.6956 - accuracy: 0.4024 - val_loss: 0.6959 - val_accuracy: 0.3750
Epoch

Epoch 1/50
22/22 [==============================] - 2s 85ms/step - loss: 0.6556 - accuracy: 0.6331 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
22/22 [==============================] - 1s 59ms/step - loss: 0.5228 - accuracy: 0.8358 - val_loss: 0.4656 - val_accuracy: 0.8631
Epoch 3/50
22/22 [==============================] - 1s 62ms/step - loss: 0.3462 - accuracy: 0.9586 - val_loss: 0.3139 - val_accuracy: 0.9405
Epoch 4/50
22/22 [==============================] - 1s 61ms/step - loss: 0.1977 - accuracy: 0.9882 - val_loss: 0.2234 - val_accuracy: 0.9405
Epoch 5/50
22/22 [==============================] - 1s 62ms/step - loss: 0.1153 - accuracy: 0.9926 - val_loss: 0.1819 - val_accuracy: 0.9464
Epoch 6/50
22/22 [==============================] - 1s 61ms/step - loss: 0.0729 - accuracy: 0.9941 - val_loss: 0.1631 - val_accuracy: 0.9464
Epoch 7/50
22/22 [==============================] - 1s 63ms/step - loss: 0.0496 - accuracy: 0.9970 - val_loss: 0.1540 - val_accuracy: 0.9524
Epoch

Epoch 1/50
22/22 [==============================] - 2s 87ms/step - loss: 0.6672 - accuracy: 0.6065 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
22/22 [==============================] - 1s 57ms/step - loss: 0.5786 - accuracy: 0.7160 - val_loss: 0.5432 - val_accuracy: 0.7321
Epoch 3/50
22/22 [==============================] - 1s 57ms/step - loss: 0.4588 - accuracy: 0.9024 - val_loss: 0.4311 - val_accuracy: 0.9048
Epoch 4/50
22/22 [==============================] - 1s 58ms/step - loss: 0.3309 - accuracy: 0.9615 - val_loss: 0.3296 - val_accuracy: 0.9286
Epoch 5/50
22/22 [==============================] - 1s 58ms/step - loss: 0.2264 - accuracy: 0.9837 - val_loss: 0.2585 - val_accuracy: 0.9405
Epoch 6/50
22/22 [==============================] - 1s 57ms/step - loss: 0.1590 - accuracy: 0.9911 - val_loss: 0.2156 - val_accuracy: 0.9464
Epoch 7/50
22/22 [==============================] - 1s 57ms/step - loss: 0.1151 - accuracy: 0.9896 - val_loss: 0.1896 - val_accuracy: 0.9464
Epoch

Epoch 1/50
22/22 [==============================] - 2s 85ms/step - loss: 0.6836 - accuracy: 0.5902 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
22/22 [==============================] - 1s 57ms/step - loss: 0.6315 - accuracy: 0.8210 - val_loss: 0.5874 - val_accuracy: 0.8810
Epoch 3/50
22/22 [==============================] - 1s 59ms/step - loss: 0.4977 - accuracy: 0.9586 - val_loss: 0.4309 - val_accuracy: 0.9464
Epoch 4/50
22/22 [==============================] - 1s 55ms/step - loss: 0.3048 - accuracy: 0.9867 - val_loss: 0.2767 - val_accuracy: 0.9702
Epoch 5/50
22/22 [==============================] - 1s 57ms/step - loss: 0.1533 - accuracy: 0.9926 - val_loss: 0.1928 - val_accuracy: 0.9583
Epoch 6/50
22/22 [==============================] - 1s 57ms/step - loss: 0.0766 - accuracy: 0.9956 - val_loss: 0.1604 - val_accuracy: 0.9583
Epoch 7/50
22/22 [==============================] - 1s 56ms/step - loss: 0.0428 - accuracy: 0.9985 - val_loss: 0.1482 - val_accuracy: 0.9583
Epoch

...

Epoch 1/50
22/22 [==============================] - 2s 95ms/step - loss: 0.6394 - accuracy: 0.6021 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
22/22 [==============================] - 1s 66ms/step - loss: 0.4775 - accuracy: 0.7885 - val_loss: 0.4115 - val_accuracy: 0.8750
Epoch 3/50
22/22 [==============================] - 2s 70ms/step - loss: 0.2812 - accuracy: 0.9645 - val_loss: 0.2531 - val_accuracy: 0.9405
Epoch 4/50
22/22 [==============================] - 1s 65ms/step - loss: 0.1428 - accuracy: 0.9896 - val_loss: 0.1805 - val_accuracy: 0.9524
Epoch 5/50
22/22 [==============================] - 2s 70ms/step - loss: 0.0774 - accuracy: 0.9941 - val_loss: 0.1540 - val_accuracy: 0.9524
Epoch 6/50
22/22 [==============================] - 1s 67ms/step - loss: 0.0463 - accuracy: 0.9970 - val_loss: 0.1444 - val_accuracy: 0.9464
Epoch 7/50
22/22 [==============================] - 1s 67ms/step - loss: 0.0312 - accuracy: 0.9985 - val_loss: 0.1410 - val_accuracy: 0.9464
Epoch

Epoch 1/50
22/22 [==============================] - 3s 122ms/step - loss: 0.5659 - accuracy: 0.6967 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
22/22 [==============================] - 1s 62ms/step - loss: 0.1430 - accuracy: 0.9941 - val_loss: 0.6540 - val_accuracy: 0.6369
Epoch 3/50
22/22 [==============================] - 1s 61ms/step - loss: 0.0792 - accuracy: 1.0000 - val_loss: 0.6401 - val_accuracy: 0.6369
Epoch 4/50
22/22 [==============================] - 1s 57ms/step - loss: 0.0538 - accuracy: 1.0000 - val_loss: 0.6274 - val_accuracy: 0.6369
Epoch 5/50
22/22 [==============================] - 1s 59ms/step - loss: 0.0402 - accuracy: 1.0000 - val_loss: 0.6138 - val_accuracy: 0.6369
Epoch 6/50
22/22 [==============================] - 1s 58ms/step - loss: 0.0318 - accuracy: 1.0000 - val_loss: 0.6002 - val_accuracy: 0.6369
Epoch 7/50
22/22 [==============================] - 1s 62ms/step - loss: 0.0259 - accuracy: 1.0000 - val_loss: 0.5856 - val_accuracy: 0.6369
Epoc

Epoch 1/50
22/22 [==============================] - 2s 91ms/step - loss: 0.6829 - accuracy: 0.5917 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
22/22 [==============================] - 1s 55ms/step - loss: 0.6583 - accuracy: 0.6036 - val_loss: 0.6400 - val_accuracy: 0.6071
Epoch 3/50
22/22 [==============================] - 1s 58ms/step - loss: 0.6058 - accuracy: 0.6036 - val_loss: 0.5727 - val_accuracy: 0.6071
Epoch 4/50
22/22 [==============================] - 1s 55ms/step - loss: 0.5106 - accuracy: 0.6109 - val_loss: 0.4688 - val_accuracy: 0.6607
Epoch 5/50
22/22 [==============================] - 1s 52ms/step - loss: 0.3942 - accuracy: 0.7929 - val_loss: 0.3750 - val_accuracy: 0.8214
Epoch 6/50
22/22 [==============================] - 1s 59ms/step - loss: 0.2963 - accuracy: 0.9527 - val_loss: 0.2974 - val_accuracy: 0.8988
Epoch 7/50
22/22 [==============================] - 1s 57ms/step - loss: 0.2110 - accuracy: 0.9778 - val_loss: 0.2469 - val_accuracy: 0.9107
Epoch

Epoch 1/50
22/22 [==============================] - 2s 87ms/step - loss: 0.6641 - accuracy: 0.6050 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
22/22 [==============================] - 1s 53ms/step - loss: 0.5562 - accuracy: 0.6509 - val_loss: 0.4903 - val_accuracy: 0.7321
Epoch 3/50
22/22 [==============================] - 1s 55ms/step - loss: 0.3426 - accuracy: 0.9349 - val_loss: 0.2849 - val_accuracy: 0.9167
Epoch 4/50
22/22 [==============================] - 1s 57ms/step - loss: 0.1476 - accuracy: 0.9896 - val_loss: 0.1754 - val_accuracy: 0.9405
Epoch 5/50
22/22 [==============================] - 1s 54ms/step - loss: 0.0560 - accuracy: 0.9956 - val_loss: 0.1490 - val_accuracy: 0.9464
Epoch 6/50
22/22 [==============================] - 1s 57ms/step - loss: 0.0241 - accuracy: 0.9970 - val_loss: 0.1487 - val_accuracy: 0.9524
Epoch 7/50
22/22 [==============================] - 1s 57ms/step - loss: 0.0127 - accuracy: 1.0000 - val_loss: 0.1524 - val_accuracy: 0.9524
Epoch

INFO:tensorflow:Oracle triggered exit
Epoch 1/50
28/28 [==============================] - 4s 127ms/step - loss: 0.7369 - accuracy: 0.6209 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
28/28 [==============================] - 2s 65ms/step - loss: 0.6338 - accuracy: 0.6552 - val_loss: 0.6697 - val_accuracy: 0.6071
Epoch 3/50
28/28 [==============================] - 2s 64ms/step - loss: 0.5306 - accuracy: 0.7275 - val_loss: 0.6517 - val_accuracy: 0.6071
Epoch 4/50
28/28 [==============================] - 2s 64ms/step - loss: 0.4911 - accuracy: 0.7595 - val_loss: 0.6244 - val_accuracy: 0.6131
Epoch 5/50
28/28 [==============================] - 2s 64ms/step - loss: 0.4146 - accuracy: 0.8353 - val_loss: 0.5946 - val_accuracy: 0.6369
Epoch 6/50
28/28 [==============================] - 2s 64ms/step - loss: 0.3639 - accuracy: 0.8448 - val_loss: 0.5598 - val_accuracy: 0.6548
Epoch 7/50
28/28 [==============================] - 2s 70ms/step - loss: 0.2846 - accuracy: 0.9076 - val_lo

In [0]:
test_mix[0]

("www.breakingnews247.net/59bc13819a723/world-s-most-popular-candy-to-be-removed-from-shelves-by-october-2017.html, World's most popular candy to be removed from shelves by October 2017!",
 'fake')

In [0]:
test_mix[0][0]

"www.breakingnews247.net/59bc13819a723/world-s-most-popular-candy-to-be-removed-from-shelves-by-october-2017.html, World's most popular candy to be removed from shelves by October 2017!"

In [0]:
clf.predict(np.array([test_mix[0][0]]))

array([['fake']], dtype='<U4')

In [0]:
y_true = [label for x, label in test_mix]
x_test = [x for x, label in test_mix]
y_true = np.array(y_true)
x_test = np.array(x_test)

In [0]:
y_pre = clf.predict(x_test)
y_pre

array([['fake'],
       ['real'],
       ['real'],
       ['real'],
       ['fake'],
       ['real'],
       ['fake'],
       ['real'],
       ['fake'],
       ['real'],
       ['fake'],
       ['fake'],
       ['fake'],
       ['real'],
       ['fake'],
       ['fake'],
       ['fake'],
       ['real'],
       ['real'],
       ['fake'],
       ['fake'],
       ['real'],
       ['real'],
       ['real'],
       ['fake'],
       ['fake'],
       ['fake'],
       ['real'],
       ['real'],
       ['real'],
       ['real'],
       ['real'],
       ['fake'],
       ['fake'],
       ['fake'],
       ['real'],
       ['fake'],
       ['real'],
       ['real'],
       ['real'],
       ['real'],
       ['real'],
       ['real'],
       ['real'],
       ['fake'],
       ['real'],
       ['fake'],
       ['real'],
       ['fake'],
       ['fake'],
       ['real'],
       ['real'],
       ['real'],
       ['fake'],
       ['fake'],
       ['fake'],
       ['fake'],
       ['real'],
       ['real'

In [0]:
acc, precision, recall, f1 = calculate_metrics(y_true, y_pre)
print("acc: ", acc)
print("precision: ", precision)
print("recall: ", recall)
print("f1: ", f1)

acc:  0.8632075471698113
precision:  0.8108108108108109
recall:  0.9183673469387755
f1:  0.861244019138756
